# ECON 690 Python Project: Prediction Air BnB Prices in New York
## Braden Poe and Kevin Van Lieshout

### Outline Price Prediction:
1. Import necesary packages for the project
2. Download the data, analyze it, explain the variables, hot encode our categorical variables and do some feature selection based on high correlations
3. Break our data into testing and training samples
4. Run some normal ML models on the data to look at Root Mean Squared error evaluation
    - Linear Regression
    - Ridge Regression
        - Do a small analysis for the best alpha learning rate based on RMSE
    - Lasso Regression
    - Decision Tree Regression
    - Neural Net
    - Random Forest Regression
        - Run feature importance analysis for best explanation and re run the model with this new subset of data to see if it performs better
5. Reset the data and run a model that needs standardized data. We standardize the data and then run:
    - K Nearest Neighbors Regression
6. Reset the data again and now lets examine some different kinds of Emsemble models for better predictions
    - Voting Method using Lasso, Linear Regression and a Random Forest Regression
    - A Bagging Regressor using a RF Regressor as it's baseline model
    - An AdaBoost Regressor
    - A Gradient Boosting Regressor
    - A XG Boosting Regressor
7. Greedy Algorithms
    - Recursive Feature Elimination with
        - Linear Regression 
        - Lasso Regression
        - Random Forest Regressor
7. Evaluate how our models performed and which one seems optimal 

### Outline Rental Probability Prediction:
1. Clean up the data and grab the data needed for rental probaility 
2. Create dummies that are needed
3. Run the logistic regression model 
    - Analyze the confusion matrix 
4. Analyze the ROC curve for True Positives and False Positives
5. Fix the dataset to upload into a different heat map
6. Run the heat map analysis 
7. Evaluate how the models performed 

**As a warning, some of these models are pretty indepth and may be computationally expensive in terms of time.**

## 1). Import necesary packages for the project

In [9]:
pip install fiona


  Using cached https://files.pythonhosted.org/packages/be/04/31d0a6f03943b1684f32c9b861be40c1fd282468fa6bd54ddf4a774e6b0f/Fiona-1.8.13.tar.gz
    Complete output from command python setup.py egg_info:
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    
    ----------------------------------------
Note: you may need to restart the kernel to use updated packages.


Command "python setup.py egg_info" failed with error code 1 in C:\Users\KVANLI~1\AppData\Local\Temp\pip-install-r6j3ikg2\fiona\


In [7]:
##Import anything we could possibly need
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
from sklearn import metrics
%matplotlib inline
import geopandas as gpd
from matplotlib import cm
import fiona
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import timeit
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt 
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
import preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import mglearn
from sklearn.linear_model import Lasso
import xgboost as xg
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn import tree
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import math
import xgboost as xgb
import geopandas as gpd
from matplotlib import cm


Using TensorFlow backend.


ModuleNotFoundError: No module named 'geopandas'

## 2. Download the data, analyze it, explain the variables, hot encode our categorical variables and do some feature selection based on high correlations

In [ ]:
#Read in the data and drop columns that don't help us. And subset the data based on counts of air bnbs by neighborhood
df=pd.read_csv('~/Downloads/final.csv')

nbcounts=df.groupby([ "BOROUGH",'neighbourhood'])['price'].count()
df.drop(columns=['last_review','BOROUGH','Unnamed: 0','Unnamed: 0.1','id','name','host_name','host_id','latitude','longitude'],inplace=True)
nbcounts=pd.DataFrame(nbcounts)
nbcounts.rename(columns={'price':'counts'},inplace=True)

nbcounts.reset_index('neighbourhood',inplace=True)
df=pd.merge(df,nbcounts,how='left',on=['neighbourhood'])
med=df['price'].median()
std=df['price'].std()

df = df[df['counts']>= 100] 
df = df[(df['price']< med+2.56*std)&(df['price']> med-2.56*std)] 

df.drop(columns=['neighbourhood'],inplace=True)
df.shape

Analyze some heat maps of room and home prices in NYC

In [ ]:
room = pd.read_csv('bnbroom.csv',na_values="")
home = pd.read_csv('bnbhome.csv',na_values="")
fp = "nynta_19c/nynta.shp"
map_df = gpd.read_file(fp)
map_df.plot()


In [ ]:
mergedroom = map_df.set_index('NTAName').join(room.set_index('NTAName'))
mergedroom['roomprice'] = mergedroom['roomprice'].round(2)
mergedroom  = mergedroom.fillna(np.nan)
mergedhome = map_df.set_index('NTAName').join(home.set_index('NTAName'))
mergedhome['homeprice'] = mergedhome['homeprice'].round(2)
mergedhome = mergedhome.fillna(np.nan)

In [ ]:
# set the range for the choropleth
vmin, vmax = 0, 800.00
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(10, 10))
mergedhome.plot(column='homeprice', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')
ax.axis('off')

ax.set_title('Airbnb Entire Home/Apt Prices in New York City', fontdict={'fontsize': '25', 'fontweight' : '3'})

ax.annotate('Source: Kaggle NYC Open Data, 2019',xy=(0.1, .08),
            xycoords='figure fraction', horizontalalignment='left',
            verticalalignment='top', fontsize=12, color='#555555')
sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm,shrink = .65)

fig.savefig('homeheatmap.png', dpi=300)

In [ ]:
# set the range for the choropleth
vmin, vmax = 0, 800.00
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(10, 10))
mergedroom.plot(column='roomprice', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8')
ax.axis('off')

ax.set_title('Airbnb Room Prices in New York City', fontdict={'fontsize': '25', 'fontweight' : '3'})

ax.annotate('Source: Kaggle NYC Open Data, 2019',xy=(0.1, .08),
            xycoords='figure fraction', horizontalalignment='left',
            verticalalignment='top', fontsize=12, color='#555555')
sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm,shrink = .65)

fig.savefig('roomheatmap.png', dpi=300)

In [ ]:
#Remove high correlated variables that won't help us
threshold = 0.95

# Calculate correlations
corr_matrix = df.corr().abs()

# Subset to the upper triangle of correlation matrix
upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Identify names of columns with correlation above threshold
to_drop = [column for column in upper.columns if any(
    upper[column] >= threshold)]
print(
    f'There are {len(to_drop)} columns to drop with correlation > {threshold}')
print("These variables are:",to_drop)

In [ ]:
#Drop these variables due to their correlations being incredibly high and they probably don't help our model
df.drop(columns=to_drop,inplace=True)
df.shape

In [ ]:
#Figure out which columns are numerical and which are not

def get_cols_with_no_nans(df,col_type):
    '''
    Arguments :
    df : The dataframe to process
    col_type : 
          num : to only get numerical columns with no nans
          no_num : to only get nun-numerical columns with no nans
          all : to get any columns with no nans    
    '''
    if (col_type == 'num'):
        predictors = df.select_dtypes(exclude=['object'])
    elif (col_type == 'no_num'):
        predictors = df.select_dtypes(include=['object'])
    elif (col_type == 'all'):
        predictors = df
    else :
        print('Error : choose a type (num, no_num, all)')
        return 0
    cols_with_no_nans = []
    for col in predictors.columns:
        
        cols_with_no_nans.append(col)
    return cols_with_no_nans
num_cols = get_cols_with_no_nans(df , 'num')
cat_cols = get_cols_with_no_nans(df , 'no_num')

In [ ]:
#Do a little correlation analysis on the variables
corr = df.corr()
fig = plt.figure(figsize = (14,10))
sns.heatmap(corr,annot=True)

In [ ]:
#Hot encode the categorical variable room type
def oneHotEncode(df,colNames):
    for col in colNames:
        if( df[col].dtype == np.dtype('object')):
            dummies = pd.get_dummies(df[col],prefix=col)
            df = pd.concat([df,dummies],axis=1)

            #drop the encoded column
            df.drop([col],axis = 1 , inplace=True)
    return df
    

print('There were {} columns before encoding categorical features'.format(df.shape[1]))
df = oneHotEncode(df, cat_cols)


print('There are {} columns after encoding categorical features'.format(df.shape[1]))

In [ ]:
sns.set(color_codes=True)
ax=sns.distplot(df.price)
ax.set_title("Price Histogram")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.show()


## 3. Break our data into testing and training samples

In [ ]:
#Get the lists of numerical variables and prediction variables so you can subset X and Y for train/test
h=df.columns.tolist()
g=['price']
A=['price','BOROUGH','room_type_Shared room','room_type_Private room','room_type_Entire home/apt']


num_cols=list(set(num_cols)-set(A))
z=list(set(h)-set(g))
X = df[z]
y = df.price
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=0)



## 4. Run some normal ML models on the data to look at Root Mean Squared error evaluation
    


- Linear Regression

In [ ]:
# create linear regression object 
import math

t0 = timeit.default_timer()
reg = linear_model.LinearRegression()
  
# train the model using the training sets 
reg.fit(X_train, y_train) 
  

y_pred=reg.predict(X_test)
print("The RMSE was:",math.sqrt(metrics.mean_squared_error(y_test,y_pred)))
# variance score: 1 means perfect prediction 
print('Variance score: {}'.format(reg.score(X_test, y_test)))

t1 = timeit.default_timer()
total_time = t1 - t0
print ('This code took', total_time, 'seconds')  


- Ridge Regression with best alpha selection

In [ ]:

t0 = timeit.default_timer()#Search for best alphas in the ridge model
ridge = linear_model.RidgeCV(alphas = [ 0.1, 10, 20, 30])
ridge.fit(X_train, y_train)
alpha = ridge.alpha_
print("Best alpha :", alpha)

print("Try again for more precision with alphas centered around " + str(alpha))
ridge = linear_model.RidgeCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, alpha * .85, 
                          alpha * .9, alpha * .95, alpha, alpha * 1.05, alpha * 1.1, alpha * 1.15,
                          alpha * 1.25, alpha * 1.3, alpha * 1.35, alpha * 1.4], cv = 10)

ridge.fit(X_train, y_train)
alpha = ridge.alpha_
print("Best alpha :", alpha)
y_pred = ridge.predict(X_test)
print("The RMSE is: %.5f" % math.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print(ridge.score(X_test, y_test))
t1 = timeit.default_timer()
total_time = t1 - t0
print ('This code took', total_time, 'seconds') 

- Lasso Regression

In [ ]:
t0 = timeit.default_timer()
#run lasso model with .01 learning rate
lasso1 = Lasso(alpha=.01,max_iter=100000).fit(X_train, y_train)
print("Training Lasso score is : {:.2f}".format(lasso1.score(X_train,y_train)))
print("Test Lasso score is : {:.2f}".format(lasso1.score(X_test,y_test)))
print("Number of features used: {}".format(np.sum(lasso1.coef_ !=0)))
y_pred=lasso1.predict(X_test)
print("The RMSE was:",math.sqrt(metrics.mean_squared_error(y_test,y_pred)))
t1 = timeit.default_timer()


Lasso searching with cross validation and finding the best alpha to use 

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
t0 = timeit.default_timer()
lasso=Lasso()
parameters={'alpha':[1e-18,1e-16,1e-15,1e-10,1e-8,1e-4,1e-3,1e-2,1,5,10,20]}
lasso_reg=GridSearchCV(lasso,parameters,scoring='neg_mean_squared_error',cv=5)
lasso_reg.fit(X_train,y_train)
alpha=lasso_reg.best_params_
alpha=alpha['alpha']
best=lasso_reg.best_score_
lasso = Lasso(alpha=alpha,max_iter=100000).fit(X_train, y_train)
print("Training Lasso score is : {:.2f}".format(lasso.score(X_train,y_train)))
print("Test Lasso score is : {:.2f}".format(lasso.score(X_test,y_test)))
print("Number of features used: {}".format(np.sum(lasso.coef_ !=0)))
y_pred=lasso.predict(X_test)
print("The RMSE was:",math.sqrt(metrics.mean_squared_error(y_test,y_pred)))
t1 = timeit.default_timer()
total_time = t1 - t0
print ('This code took', total_time, 'seconds')

- Decision Tree Regression

In [ ]:
t0 = timeit.default_timer()
#Run decision tree with max of 4 levels
tree1=DecisionTreeRegressor(max_depth=4,random_state=0).fit(X_train,y_train)
print("Train:", tree1.score(X_train,y_train))
print("Test:", tree1.score(X_test,y_test))
y_pred=tree1.predict(X_test)
print("The RMSE was:",math.sqrt(metrics.mean_squared_error(y_test,y_pred)))
t1 = timeit.default_timer()
total_time = t1 - t0
print ('This code took', total_time, 'seconds')

- Neural Net Model

In [ ]:

import tensorflow as tf
from tensorflow.python.data import Dataset
import keras
from keras.utils import to_categorical
from keras import models
from keras import layers
model = keras.Sequential([
 keras.layers.Dense(64, activation=tf.nn.relu,                  
 input_shape=(X_train.shape[1],)),
 keras.layers.Dense(64, activation=tf.nn.relu),
 keras.layers.Dense(12000, activation=  'softmax')
 ])

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history2 = model.fit(
 X_train, y_train,
 epochs= 26, batch_size = 60,
 validation_data = (X_test, y_test))
y_pred=model.predict(X_test)
print("The RMSE was:",math.sqrt(math.sqrt(metrics.mean_squared_error(y_test,y_pred)))

- Random Forest Regression

In [ ]:
t0 = timeit.default_timer()
#Run Random Forest Regressor
forest=RandomForestRegressor(random_state=1)
forest.fit(X_train,y_train)
print("Training score accuracy is : {:.2f}".format(forest.score(X_train,y_train)))
print("Test score accuracy is : {:.2f}".format(forest.score(X_test,y_test)))
y_pred=forest.predict(X_test)
print("The RMSE was:",math.sqrt(metrics.mean_squared_error(y_test,y_pred)))
t1 = timeit.default_timer()
total_time = t1 - t0
print ('This code took', total_time, 'seconds')
#Plot the feature importance from the forest model since it will help us. 
def plot_feature_importances_cancer(model):
    n_features = X.shape[1]
    plt.barh(range(n_features),model.feature_importances_,align='center')
    plt.yticks(np.arange(n_features),X.columns)
    plt.xlabel("Feature Importance")
    plt.ylabel("Feature")
    plt.ylim(-1,n_features)

plot_feature_importances_cancer(forest)

- Find the most import features based on criteria above and in the selection model

In [ ]:
#Reset the data to the what the importance feature exercise showed us
t0 = timeit.default_timer()
sel = SelectFromModel(RandomForestRegressor())
sel.fit(X_train, y_train)
selected_feat= X_train.columns[(sel.get_support())]
print("There are",len(selected_feat),"important features.")
print(selected_feat)
pd.Series(sel.estimator_.feature_importances_.ravel()).hist()
X_train=X_train.filter(items=selected_feat)
X_test=X_test.filter(items=selected_feat)
t1 = timeit.default_timer()
forest=RandomForestRegressor(random_state=1)
forest.fit(X_train,y_train)
print("Training score accuracy is : {:.2f}".format(forest.score(X_train,y_train)))
print("Test score accuracy is : {:.2f}".format(forest.score(X_test,y_test)))
y_pred=forest.predict(X_test)
print("The new RMSE was:",math.sqrt(metrics.mean_squared_error(y_test,y_pred)))
total_time = t1 - t0
print ('This code took', total_time, 'seconds')

## 5. Reset the data and run a model that needs standardized data. We standardize the data and then run:
    - K Nearest Neighbors Regression

In [ ]:
#Reset the data
h=df.columns.tolist()
g=['price']
A=['price','BOROUGH','room_type_Shared room','room_type_Private room','room_type_Entire home/apt']


num_cols=list(set(num_cols)-set(A))
z=list(set(h)-set(g))
X = df[z]
y = df.price
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=0)



In [ ]:
#Normalize the X for KNN
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.

scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)


In [ ]:
#Run a nearest neighbors function 

from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=12)

knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
print("Training score accuracy is : {:.2f}".format(knn.score(X_train,y_train)))
print("Test score accuracy is : {:.2f}".format(knn.score(X_test,y_test)))
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
rmse = two_features_mse ** (1/2)
print("The RMSE was:"rmse)

## 6. Reset the data again and now lets examine some different kinds of Emsemble models for better predictions

In [ ]:
#Reset the data from being standardized with the same random_state
h=df.columns.tolist()
g=['price']
A=['price','BOROUGH','room_type_Shared room','room_type_Private room','room_type_Entire home/apt']


num_cols=list(set(num_cols)-set(A))
z=list(set(h)-set(g))
X = df[z]
y = df.price
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=0)

- Voting Method using Lasso, Linear Regression and a Random Forest Regression

In [ ]:
model1 = RandomForestRegressor()
model2 = Lasso()
model3= LinearRegression()

model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)

pred1=model1.predict(X_test)
pred2=model2.predict(X_test)
pred3=model3.predict(X_test)

final_pred = np.array([])
for i in range(0,len(X_test)):
    final_pred = np.append(final_pred, np.mean([pred1[i], pred2[i], pred3[i]]))
rmse=math.sqrt(metrics.mean_squared_error(y_test,final_pred))
print("The RMSE was:",rmse)

- Bagging regressor with a random forest regressor as its baseline model

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn import tree
model = BaggingRegressor(RandomForestRegressor())
model.fit(X_train, y_train)
print("Training score accuracy is : {:.2f}".format(model.score(X_train,y_train)))
print("Test score accuracy is : {:.2f}".format(model.score(X_test,y_test)))
final_pred=model.predict(X_test)
rmse=math.sqrt(metrics.mean_squared_error(y_test,final_pred))
print("The RMSE was:",rmse)

- Ada Boosting Regressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor()
model.fit(X_train, y_train)
model.score(X_test,y_test)
print("Training score accuracy is : {:.2f}".format(model.score(X_train,y_train)))
print("Test score accuracy is : {:.2f}".format(model.score(X_test,y_test)))
final_pred=model.predict(X_test)
rmse=math.sqrt(metrics.mean_squared_error(y_test,final_pred))
print("The RMSE was:",rmse)

- Gradient Boosting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model= GradientBoostingRegressor()
model.fit(X_train, y_train)
model.score(X_test,y_test)
print("Training score accuracy is : {:.2f}".format(model.score(X_train,y_train)))
print("Test score accuracy is : {:.2f}".format(model.score(X_test,y_test)))
final_pred=model.predict(X_test)
rmse=math.sqrt(metrics.mean_squared_error(y_test,final_pred))
print("The RMSE was:",rmse)

- XGBoost Regression

In [ ]:
import xgboost as xgb
model=xgb.XGBRegressor()
model.fit(X_train, y_train)
model.score(X_test,y_test)
print("Training score accuracy is : {:.2f}".format(model.score(X_train,y_train)))
print("Test score accuracy is : {:.2f}".format(model.score(X_test,y_test)))
final_pred=model.predict(X_test)
rmse=math.sqrt(metrics.mean_squared_error(y_test,final_pred))
print("The RMSE was:",rmse)

## 7. Greedy Algorithms


- RFE feature selection with Linear Regression

In [ ]:

t0 = timeit.default_timer()
from sklearn.feature_selection import RFE
nof_list = np.arange(1,20)
best_score=120
#Variable to store in the optimum features
nof=0
score_list=[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    model=LinearRegression()
    rfe=RFE(model,nof_list[n])
    X_train_rfe=rfe.fit_transform(X_train,y_train)
    X_test_rfe=rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    y_pred=model.predict(X_test_rfe)
    score= math.sqrt(metrics.mean_squared_error(y_pred,y_test))
    score_list.append(score)
    print(score)
    if(score<best_score):
        best_score=score

        nof=nof_list[n]  

print("Optimum number of features for the Linear Reg was: %d" %nof)
print("Score with %d features: %f" % (nof, best_score))

t1 = timeit.default_timer()
total_time = t1 - t0
print ('This code took', total_time, 'seconds') 


- RFE feature selection with Lasso Regression

In [ ]:

t0 = timeit.default_timer()
from sklearn.feature_selection import RFE
nof_list = np.arange(1,20)
best_score=120
#Variable to store in the optimum features
nof=0
score_list=[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    model=Lasso()
    rfe=RFE(model,nof_list[n])
    X_train_rfe=rfe.fit_transform(X_train,y_train)
    X_test_rfe=rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    y_pred=model.predict(X_test_rfe)
    score= math.sqrt(metrics.mean_squared_error(y_pred,y_test))
    score_list.append(score)
    print(score)
    if(score<best_score):
        best_score=score

        nof=nof_list[n]  

print("Optimum number of features for the Lasso regression was: %d" %nof)
print("Score with %d features: %f" % (nof, best_score))

t1 = timeit.default_timer()
total_time = t1 - t0
print ('This code took', total_time, 'seconds') 

- RFE feature selection with Random Forest Regression

In [ ]:

t0 = timeit.default_timer()
from sklearn.feature_selection import RFE
nof_list = np.arange(1,20)
best_score=120
#Variable to store in the optimum features
nof=0
score_list=[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    model=RandomForestRegressor()
    rfe=RFE(model,nof_list[n])
    X_train_rfe=rfe.fit_transform(X_train,y_train)
    X_test_rfe=rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    y_pred=model.predict(X_test_rfe)
    score= math.sqrt(metrics.mean_squared_error(y_pred,y_test))
    score_list.append(score)
    print(score)
    if(score<best_score):
        best_score=score

        nof=nof_list[n]  

print("Optimum number of features for the random forest regression was: %d" %nof)
print("Score with %d features: %f" % (nof, best_score))

t1 = timeit.default_timer()
total_time = t1 - t0
print ('This code took', total_time, 'seconds') 

## 7. Evaluate how our models performed and which one seems optimal 

After investigation, there were models that performed pretty well, but it turns out that the use of ensemble methods in bagging helps our model perform the best. The Bagging ensemble method mixed with the use of a random forest had the best test root mean squared error of all of the models

# RENTAL PROBABILITY PREDICTION


## 1. Clean up the data and grab the data needed for rental probaility

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib

In [ ]:
data = pd.read_csv('updated.csv',na_values="")
data = data.dropna()
data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'],inplace=True)
data = data.loc[data['minimum_nights']<=3] #  Drop minimum nights that are greater than 1/2 a week 

cols = ['price','minimum_nights','number_of_reviews','availability_365','walk_score','transit_score','bike_score','BOROUGH','SALE PRICE','crimes','gdp_cap','rest','pop_dens','r_censor','room']
y = data['r_censor']
X = data[cols]
X.to_csv('trimmed.csv')
X = X.drop('r_censor',axis=1)
X['room'] = X['room'].astype(int)
X['minimum_nights'] = X['minimum_nights'].astype(int)
X['BOROUGH'] = X['BOROUGH'].astype(int)
X = X.rename(columns={'SALE PRICE': 'sale_price'})

## 2. Create dummies that are needed

In [ ]:
# Create dummies
## At this point, the manipulation of our main datasets is complete.
X['min_night1'] = 0
X['min_night1'].loc[X['minimum_nights']==1] = 1
X['min_night2'] = 0
X['min_night2'].loc[X['minimum_nights']==2] = 2
X['min_night3'] = 0
X['min_night3'].loc[X['minimum_nights']==3] = 3
X['boro1'] = 0
X['boro1'].loc[X['BOROUGH']==1] = 1
X['boro2'] = 0
X['boro2'].loc[X['BOROUGH']==2] = 2
X['boro3'] = 0
X['boro3'].loc[X['BOROUGH']==3] = 3
X['boro4'] = 0
X['boro4'].loc[X['BOROUGH']==4] = 4
X['boro5'] = 0
X['boro5'].loc[X['BOROUGH']==5] = 5
X['room0'] = 0
X['room0'].loc[X['room']==0] = 0
X['room1'] = 0
X['room1'].loc[X['room']==1] = 1
X['room2'] = 0
X['room2'].loc[X['room']==2] = 2
X = X.drop(['BOROUGH','minimum_nights','room'],axis=1)
# Standardize certain columns 
cols_to_stan = ['price','number_of_reviews','availability_365','walk_score','transit_score','bike_score','sale_price','crimes','gdp_cap','rest','pop_dens']

for i in cols_to_stan:
    X[i] = (X[i] - X[i].mean())/X[i].std()
X.head()

In [ ]:
## Train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))
y_train = pd.DataFrame(y_train)
y_train['r_censor'] = y_train['r_censor'].astype(int)
y_test = pd.DataFrame(y_test)
y_test['r_censor'] = y_test['r_censor'].astype(int)

## 3. Run the logistic regression model

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('The MSE on the test sample is: {0}.'.format(round(mean_squared_error(y_test,y_pred),4)))
score = logreg.score(X_test,y_test)
print('The score of the model is: {0}.'.format(round(score,4)))
y_pred = pd.DataFrame(y_pred)

- Look at the confusion matrix

In [ ]:
## Confusion Matrix 
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

## 4. Analyze the ROC curve for True Positives and False Positives

In [ ]:
##ROC Curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
X['pred'] = logreg.predict(X)
X.head()
X.shape


## 5. Fix the data up for heat map analysis

In [ ]:
## Load original bnb data set 
bnb = pd.read_csv('final.csv')

## Make equivalent modifications
bnb = bnb.loc[bnb['minimum_nights']<=3]
bnb = bnb.dropna()
bnb.head()
## Merge datasets in order to create heatmap
merge = bnb.merge(X['pred'],left_index=True,right_index=True)
merge.rename(columns={'neighbourhood':'NTAName'},inplace=True)
merge.head()

In [ ]:
## Load heatmap 
fp = "nynta.shp"
map_df = gpd.read_file(fp)
map_df.head()

heatmap = pd.read_csv('heatmap.csv')
# Merge 
heat = map_df.set_index('NTAName').join(heatmap.set_index('NTAName'))
heat  = heat.fillna(0)
heat.head()

## 6. Heat map analysis

In [2]:
## Rental heat maps
# set the range for the choropleth
vmin, vmax = 0, 1
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(10, 10))
heat.plot(column='pred', cmap='Greens', linewidth=0.8, ax=ax, edgecolor='0.8')
ax.axis('off')

ax.set_title('Airbnb Rental Probability\nby Neighbourhood', fontdict={'fontsize': '25', 'fontweight' : '2'})

ax.annotate('Source: Kaggle NYC Open Data, 2019',xy=(0.1, .08),
            xycoords='figure fraction', horizontalalignment='left',
            verticalalignment='top', fontsize=12, color='#555555')
sm = plt.cm.ScalarMappable(cmap='Greens', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm,shrink = .65)

fig.savefig('rentalprobheat.png', dpi=300)

NameError: name 'plt' is not defined

## 7. Evaluate how the model performed

The model performed pretty well, the confusion matrix has more missed classifications than we would have liked, but the ROC curve looks okay and we showed good power in the logistic regression